In [65]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [1]:
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')
df = pd.read_sql_query("SELECT * from Fires", conn)

In [5]:
df[10000:10010]

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
10000,1596775,201615000,SFO-MN-2012-2012312167,NONFED,ST-NASF,ST/C&L,USMNMNS,Minnesota Department of Natural Resources,MNDNR,Minnesota Department of Natural Resources,...,A,45.869652,-93.192234,8.0,PRIVATE,MN,Kanabec,065,Kanabec,b'\x00\x01\xad\x10\x00\x00\x10f\xd2\x8dMLW\xc0...
10001,1248030,1651189,SFO-GA-FY2001-Paulding-035,NONFED,ST-NASF,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,Georgia Forestry Commission,...,A,33.795800,-84.754400,8.0,PRIVATE,GA,Paulding,223,Paulding,b'\x00\x01\xad\x10\x00\x00\x8c\x06\xf0\x16H0U\...
10002,1357657,1838687,SFO-TN-2007-900,NONFED,ST-NASF,ST/C&L,USTNTNS,Tennessee Division of Forestry,TDA,Tennessee Department of Agriculture,...,C,35.673333,-85.186670,8.0,PRIVATE,TN,BLEDSOE,007,Bledsoe,b'\x00\x01\xad\x10\x00\x008I\xbaf\xf2KU\xc0\xb...
10003,422677,456674,SFO-GA00680505-36-175-0112-03,NONFED,ST-NASF,ST/C&L,USGAGAS,Georgia Forestry Commission,GA McRae,"GAS Ogeechee District, McRae Office",...,B,32.488140,-82.970134,14.0,MISSING/NOT SPECIFIED,GA,Laurens,175,Laurens,b'\x00\x01\xad\x10\x00\x008\xf5\xab\xad\x16\xb...
10004,1327749,1806757,SFO-NY-NY0724-2003-001058,NONFED,ST-NASF,ST/C&L,USNYNYX,Fire Department of New York,NY0724,Fire Department of New York,...,B,42.082938,-79.253567,14.0,MISSING/NOT SPECIFIED,NY,CHAUTAUQUA,013,Chautauqua,b'\x00\x01\xad\x10\x00\x00X\x88\xedr:\xd0S\xc0...
10005,917743,1044991,SWRA_SC_58622,NONFED,ST-SCSCS,ST/C&L,USSCSCS,South Carolina Forestry Commission,SCSCS4,SCS Unit 4,...,A,33.633300,-79.900000,14.0,MISSING/NOT SPECIFIED,SC,None,None,None,b'\x00\x01\xad\x10\x00\x00\x98\x99\x99\x99\x99...
10006,1261787,1665124,SFO-GA-FY2003-McDuffie-010,NONFED,ST-NASF,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,Georgia Forestry Commission,...,A,33.474749,-82.554078,14.0,MISSING/NOT SPECIFIED,GA,McDuffie,189,McDuffie,b'\x00\x01\xad\x10\x00\x00\x98hg\x03v\xa3T\xc0...
10007,323187,330043,W-121736,FED,DOI-WFMI,BLM,USCOCRD,Northwest District,COCRD,"Western Slope Center, Craig",...,B,40.259700,-107.415100,7.0,STATE,CO,Routt,107,Routt,b'\x00\x01\xad\x10\x00\x00p$\x97\xff\x90\xdaZ\...
10008,1110996,1352971,CDF_1997_54_2215_10563,NONFED,ST-CACDF,ST/C&L,USCABTU,Butte Unit,CABTU,Butte Unit,...,B,39.428889,-121.715000,14.0,MISSING/NOT SPECIFIED,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\xf4(\\\x8f\xc2m^\xc...
10009,729301,828198,SC_5892,NONFED,ST-SCSCS,ST/C&L,USSCSCS,South Carolina Forestry Commission,SCSCS,South Carolina Forestry Commission,...,B,33.129167,-80.304167,14.0,MISSING/NOT SPECIFIED,SC,18,035,Dorchester,b'\x00\x01\xad\x10\x00\x00\xb8\x0b{ww\x13T\xc0...


In [53]:
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df[['STATE']])
print(ohe.categories_)
df[ohe.categories_[0]] = transformed.toarray()

[array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN',
       'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)]


In [67]:
X = df[["DISCOVERY_DOY", "FIRE_SIZE" ,"LONGITUDE", "LATITUDE"]]
y = df["STAT_CAUSE_CODE"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [68]:
from sklearn.ensemble import RandomForestClassifier
rfr = RandomForestClassifier().fit(X_train, y_train)

In [69]:
y_pred = rfr.predict(X_test)

In [70]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.70      0.80      0.75     30850
         2.0       0.32      0.23      0.26     16336
         3.0       0.16      0.04      0.07      5821
         4.0       0.40      0.24      0.30      8360
         5.0       0.47      0.63      0.53     47034
         6.0       0.36      0.26      0.30      3741
         7.0       0.46      0.45      0.45     30747
         8.0       0.25      0.11      0.16      6746
         9.0       0.44      0.45      0.44     35709
        10.0       0.55      0.41      0.47      1303
        11.0       0.11      0.02      0.04      1535
        12.0       0.15      0.03      0.05       409
        13.0       0.57      0.51      0.54     18261

    accuracy                           0.49    206852
   macro avg       0.38      0.32      0.34    206852
weighted avg       0.47      0.49      0.47    206852

[[24760   555    87   572  1410    98   651    82  1839    41    28     3
    7

In [14]:
df.groupby(["STATE", "STAT_CAUSE_DESCR"])["STAT_CAUSE_CODE"].count()

STATE  STAT_CAUSE_DESCR 
AK     Arson                150
       Campfire             499
       Children             186
       Debris Burning       777
       Equipment Use        137
                           ... 
WY     Missing/Undefined    709
       Powerline             42
       Railroad             131
       Smoking              100
       Structure              3
Name: STAT_CAUSE_CODE, Length: 623, dtype: int64

In [16]:
df.groupby(["STAT_CAUSE_DESCR", "STATE"])["STAT_CAUSE_DESCR"].count()

STAT_CAUSE_DESCR  STATE
Arson             AK        150
                  AL       7244
                  AR       3384
                  AZ       1791
                  CA       6628
                           ... 
Structure         VA         38
                  WA         44
                  WI        118
                  WV        111
                  WY          3
Name: STAT_CAUSE_DESCR, Length: 623, dtype: int64

In [27]:
df.groupby("STAT_CAUSE_DESCR").mean()["FIRE_SIZE"]

STAT_CAUSE_DESCR
Arson                 30.676274
Campfire              37.324330
Children               7.371390
Debris Burning        14.719021
Equipment Use         37.892808
Fireworks             23.820233
Lightning            295.441598
Miscellaneous         47.604008
Missing/Undefined     40.996310
Powerline             80.448005
Railroad              24.329105
Smoking               13.336670
Structure             35.595463
Name: FIRE_SIZE, dtype: float64

In [43]:
lightning = df[df["STAT_CAUSE_DESCR"] == "Lightning"]
lightning["FIRE_SIZE"]

22         0.20
27        15.00
32         0.10
36         0.10
47         0.50
          ...  
626772     0.10
626778     0.10
626779     0.20
626790     2.50
626818     0.01
Name: FIRE_SIZE, Length: 93057, dtype: float64